# Ensemble best models

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

# Load model
MODEL_PATH_2= "results/regularization/2_layer/01_10_acc_es_128_l2_9_bn_256_l2_9_bn_conv_1024_dout_08_512_dense/pt_80_10_10_lr_0.0001_bs_64_ep_100/model.h5"
model_2 = tf.keras.models.load_model(MODEL_PATH_2)

MODEL_PATH_3= "results/regularization/3_layer/01_20_acc_es_32_l2_9_bn_64_l2_9_bn_128_l2_9_bn_nopool_conv_1024_dout_09_512_dense/pt_80_10_10_lr_0.0001_bs_64_ep_200/model.h5"
model_3 = tf.keras.models.load_model(MODEL_PATH_3)

def soft_voting(ls_models, X, n_out=101):
    y = np.zeros((X.shape[0], n_out))
    for model in ls_models:
        y += model.predict(X)
    return np.argmax(y, axis=1)


2023-11-17 16:48:22.648025: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-17 16:48:22.671777: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-17 16:48:23.079484: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-11-17 16:48:25.080950: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2023-11-17 16:48:25.080983: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:168] retrievin

In [2]:
from src.dataloader import read_train, read_val, read_test
X_train, y_train = read_train("input/data/train_80_10_10/")
X_val, y_val = read_val("input/data/train_80_10_10/")
X_test, y_test = read_test("input/data/train_80_10_10/")

In [9]:
# EVALUATE
print("2 LAYERS TRAIN: ", model_2.evaluate(X_train, y_train, verbose=0)[1])
print("3 LAYERS TRAIN: ", model_3.evaluate(X_train, y_train, verbose=0)[1])

print("2 LAYERS VALID: ", model_2.evaluate(X_val, y_val, verbose=0)[1])
print("3 LAYERS VALID: ", model_3.evaluate(X_val, y_val, verbose=0)[1])

print("2 LAYERS TEST: ", model_2.evaluate(X_test, y_test, verbose=0)[1])
print("3 LAYERS TEST: ", model_3.evaluate(X_test, y_test, verbose=0)[1])

2 LAYERS TRAIN:  0.9251837730407715
3 LAYERS TRAIN:  0.8440247774124146
2 LAYERS VALID:  0.7797001004219055
3 LAYERS VALID:  0.7600922584533691
2 LAYERS TEST:  0.7289503812789917
3 LAYERS TEST:  0.7151095867156982


In [10]:
y_pred_train = soft_voting([model_2, model_3], X_train)
print("ENSEMBLE TRAIN: ", np.mean(y_pred_train == y_train))

y_pred_val = soft_voting([model_2, model_3], X_val)
print("ENSEMBLE VAL: ", np.mean(y_pred_val == y_val))

y_pred = soft_voting([model_2, model_3], X_test)
print("ENSEMBLE TEST: ", np.mean(y_pred == y_test))

217/217 [==============================] - 1s 3ms/step
ENSEMBLE TRAIN:  0.9150929796742108
28/28 [==============================] - 0s 2ms/step
ENSEMBLE VAL:  0.7797001153402537
28/28 [==============================] - 0s 2ms/step
ENSEMBLE TEST:  0.734717416378316


In [8]:
y_pred = soft_voting([model_2, model_3], X_test)
print("ENSEMBLE TEST: ", np.mean(y_pred == y_test))

28/28 [==============================] - 0s 4ms/step
ENSEMBLE TEST:  0.734717416378316
